<a href="https://colab.research.google.com/github/aisyah202255202052/Aisyah_TugasMachineLearning/blob/main/Pedeteksi_serangan_DDoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Besar Networking Security
## Analisis Serangan DDoS menggunakan pendekatan Machine Learning menggunakan Model RandomForestClassfier

### Kelompok 5
- 1204041 - Bryan Samperura
- 1204037 - Muhammad Safwan Suhadi

### Kelas & Prodi
- 3B - D4 Teknik Informatika

### Dosen Pengampu
- Diash Firdaus

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, MiniBatchKMeans
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import seaborn as sns

In [3]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Aisyah/TM9/Dataset/2021-09-27_Flow.csv")
df1 = pd.read_csv("/content/drive/MyDrive/Aisyah/TM9/Dataset/2021-09-29_Flow.csv")

df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,No Label
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,No Label
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,No Label


In [4]:
frames = [df, df1]
dfr = pd.concat(frames).drop_duplicates().reset_index(drop=True)
dfr.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,No Label
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,No Label
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,No Label


In [5]:
dfr.rename(columns = {' Label':'Label'}, inplace= True)
dfr.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,No Label
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,No Label
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,No Label


In [6]:
dfr['Label'].value_counts()

Label
No Label    1509
Name: count, dtype: int64

In [7]:
dfr.Label = [0 if each == 'BENIGN' else 1 for each in dfr.Label]
dfr.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,1
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,1
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,1


In [8]:
dfr['Label'].value_counts()

Label
1    1509
Name: count, dtype: int64

In [9]:
print(dfr.columns)

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE 

In [11]:
le = preprocessing.LabelEncoder()
X1 = dfr.apply(le.fit_transform)
X1

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,86,13,3,102,51,1,5,46,0,1,...,0,0,0,0,0,0,0,0,0,0
1,371,30,358,248,4,1,10,251,80,112,...,0,0,0,0,0,0,0,0,0,0
2,258,30,86,172,4,1,6,371,1,2,...,0,126,0,117,415,226,0,137,356,0
3,31,30,309,38,4,1,11,374,1,2,...,0,35,0,32,78,233,0,144,421,0
4,492,30,14,134,4,1,0,441,2,3,...,0,94,58,90,289,421,829,779,367,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504,295,30,439,200,4,1,916,497,12,18,...,0,767,609,754,649,385,806,392,351,0
1505,294,30,219,200,4,1,915,502,16,22,...,0,794,615,761,648,386,808,395,350,0
1506,294,30,219,200,4,1,933,20,1,0,...,0,0,0,0,0,0,0,0,0,0
1507,365,30,274,246,4,1,915,501,15,22,...,0,743,604,748,577,391,812,400,346,0


In [19]:
Y1 = dfr['Label'].values
Y1[0:5]

array([1, 1, 1, 1, 1])

In [20]:
X2 = preprocessing.StandardScaler().fit(X1).transform(X1.astype('float64'))
X2[0:2]

array([[-1.00178726, -2.06576315, -1.11653116, -0.35990733,  2.95821871,
         0.14729051, -1.50968273, -1.54930102, -0.61122553, -0.55264365,
        -0.74044617, -0.58426301, -0.72461647, -0.28443112, -0.82021025,
        -0.72893523, -0.38585323, -0.35702976, -0.31094336,  0.11000942,
         1.7330309 ,  1.55070909, -1.54783242, -1.28421596, -1.55215757,
        -0.18487202, -1.13628199, -1.13498192, -1.00448296, -1.13601189,
        -0.77389841, -1.01068584, -1.00995117, -0.91100068, -1.01096452,
        -0.13445309,  0.        ,  1.49838623,  0.        ,  0.        ,
        -0.72181557, -0.65078842, -1.51454882,  1.60944034, -0.26969341,
        -0.49630658, -0.04714456, -0.17670883, -0.17731513, -0.28314625,
        -0.48462448,  0.        ,  1.49838623,  0.70499916,  0.        ,
         0.        ,  0.        , -1.18774288,  0.23616872, -0.81562022,
        -0.30509402,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.61122553, -0.7

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X2, Y1, test_size=0.2, random_state=0)
print('Train set: ', X_train.shape, Y_train.shape)
print('Test set: ', X_test.shape, Y_test.shape)

Train set:  (1207, 84) (1207,)
Test set:  (302, 84) (302,)


In [22]:
model = RandomForestClassifier(n_estimators=5)
model.fit(X_train, Y_train)
y_val_pred=model.predict(X_test)
y_val_pred=pd.DataFrame(y_val_pred)
print("Akurasi Skor :", (accuracy_score(y_val_pred,Y_test)*100,"%"))

Akurasi Skor : (100.0, '%')


# Dataset Realtime Serangan Normal & Serangan DDoS
 - Pendekatan Model Machine Learning dengan algoritma Random Forest
 - Dataset Serangan Normal & Serangan DDoS selama 3 Menit
 - Data Realtime dengan menggunakan TCPDump dengan Tools extract CICFLOWMETER
 - Menggunakan Topologi 5 Host, 1 Switch, dengan masing masing Bandwith 10 mbps
 - Melakukan pendekatan data untuk mencari Algoritma terbaik menggunakan Dataaset CICIDS2017

# Membaca Dataset CSV

In [24]:
df_s = pd.read_csv("/content/drive/MyDrive/Aisyah/TM9/Dataset/2021-09-27_Flow.csv")
df_n = pd.read_csv("/content/drive/MyDrive/Aisyah/TM9/Dataset/2021-09-27_Flow.csv")

df_s.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,No Label
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,No Label
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,No Label
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,No Label


# Melakukan Analisis Dataset & Merger
 - No Label pada Serangan Normal dibuat menjadi 0
 - No Label pada serangan DDoS dibuat menjadi 1
 - Lalu melakukan pengabungan Dataset

In [25]:
df_s['Label'].value_counts()

Label
No Label    1254
Name: count, dtype: int64

In [26]:
df_s.Label = [0 if each == 'No_Label' else 1 for each in df_s.Label]
df_s.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,1
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,1
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,1


In [27]:
df_n.Label = [1 if each == 'No_Label' else 0 for each in df_n.Label]
df_n.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,0
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,0
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,0


In [28]:
frames = [df_s, df_n]
dfa = pd.concat(frames).drop_duplicates().reset_index(drop=True)
dfa.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,142.250.194.35-172.16.81.146-443-59925-6,142.250.194.35,443,172.16.81.146,59925,6,27/09/2021 02:32:44 pm,385,0,2,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1
1,172.16.81.146-74.125.158.105-63316-443-6,172.16.81.146,63316,74.125.158.105,443,6,27/09/2021 02:32:53 pm,1935459,115,282,...,0,0.0,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1
2,172.16.81.146-216.58.200.168-51675-443-6,172.16.81.146,51675,216.58.200.168,443,6,27/09/2021 02:32:48 pm,14731103,1,3,...,0,52467.0,0.000000,52467.0,52467.0,14678263.0,0.000000e+00,14678263.0,14678263.0,1
3,142.250.192.238-172.16.81.146-443-60922-6,172.16.81.146,60922,142.250.192.238,443,6,27/09/2021 02:32:54 pm,14771324,1,3,...,0,36839.0,0.000000,36839.0,36839.0,14734086.0,0.000000e+00,14734086.0,14734086.0,1
4,172.217.167.3-172.16.81.146-443-49334-6,172.16.81.146,49334,172.217.167.3,443,6,27/09/2021 02:32:32 pm,59792976,2,4,...,0,39971.0,212.132034,40121.0,39821.0,29856217.0,2.143562e+07,45013492.0,14698942.0,1


In [29]:
dfa['Label'].value_counts()

Label
1    1254
0    1254
Name: count, dtype: int64

# Menentukan Input & Output
- Tentukan Input pada Dataset yang akan di Train & Test
- Tentukan Output pada Dataset yang akan di Train & Test
- Merubah Dataset menjadi Normal

In [30]:
print(dfa.columns)

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE 

In [31]:
X1 = dfa [['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
       'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
       'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
       'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
       'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
       'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']]

In [32]:
le = preprocessing.LabelEncoder()
X1 = dfa.apply(le.fit_transform)
X1

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,60,10,3,68,35,1,5,28,0,1,...,0,0,0,0,0,0,0,0,0,1
1,204,24,222,162,4,1,10,134,76,109,...,0,0,0,0,0,0,0,0,0,1
2,157,24,40,123,4,1,6,204,1,2,...,0,122,0,115,387,212,0,123,337,1
3,25,24,189,20,4,1,11,207,1,2,...,0,32,0,30,69,219,0,130,402,1
4,298,24,11,95,4,1,0,265,2,3,...,0,91,58,88,267,394,775,708,348,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,226,24,22,170,7,1,851,504,2,3,...,0,151,60,141,423,691,173,596,674,0
2504,121,23,0,135,0,0,857,914,11,0,...,0,0,0,0,0,33,158,9,228,0
2505,280,24,26,82,4,1,853,584,11,16,...,0,404,366,429,295,741,89,599,760,0
2506,33,24,159,24,4,1,859,818,24,35,...,0,763,400,633,783,254,350,174,470,0


In [33]:
Y1 = dfa['Label'].values
Y1[0:5]

array([1, 1, 1, 1, 1])

In [34]:
X2 = preprocessing.StandardScaler().fit(X1).transform(X1.astype('float64'))
X2[0:2]

array([[-0.94735898, -2.16644721, -1.0509752 , -0.35890668,  2.88404968,
         0.15542481, -1.43916209, -1.59607035, -0.65957717, -0.57968559,
        -0.76849532, -0.59584065, -0.73626869, -0.30509726, -0.85289392,
        -0.74206336, -0.36822284, -0.38890906, -0.30252639,  0.2030147 ,
         1.7215112 ,  1.59896037, -1.59751476, -1.36047064, -1.59956958,
        -0.18647004, -1.24263468, -1.23924884, -1.08297679, -1.24232302,
        -0.84606854, -0.98687144, -0.98692572, -0.94584189, -0.98721327,
        -0.05458116,  0.        ,  1.34832397,  0.        ,  0.        ,
        -0.77568978, -0.66576674, -1.60342442,  1.63996073, -0.2966361 ,
        -0.47819526,  0.0073325 , -0.12657113, -0.12692588, -0.21620494,
        -0.42388695,  0.        ,  1.34832397,  0.68564241,  0.        ,
         0.        ,  0.        , -1.19418588,  0.32437957, -0.84631944,
        -0.29538659,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.65957717, -0.7

# Train & Test Dataset

- Melakukan Train dan Test dari 20% yang ditest dan 80% yang ditrain
- Menentukan Model Machine Learning
- Mendapatkan Akurasi Skor Dataset

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X2, Y1, test_size=0.2, random_state=0)
print('Train set: ', X_train.shape, Y_train.shape)
print('Test set: ', X_test.shape, Y_test.shape)

Train set:  (2006, 84) (2006,)
Test set:  (502, 84) (502,)


In [36]:
model = RandomForestClassifier(n_estimators=5)
model.fit(X_train, Y_train)
y_val_pred=model.predict(X_test)
y_val_pred=pd.DataFrame(y_val_pred)
print("Akurasi Skor :", (accuracy_score(y_val_pred,Y_test)*100,"%"))

Akurasi Skor : (93.02788844621513, '%')


# Kesimpulan
- Algoritma yang dipilih adalah RandomForestClassifier karena akurasinya 100%